In [182]:
from src import models
import torch
import audiofile
from IPython.display import Audio, display ,Video
import time
import subprocess
import os
import time




In [184]:
class MakeAudio:
    def __init__(self,video_path):
        if torch.cuda.is_available(): self.device = "cuda"
        elif torch.backends.mps.is_available(): self.device = torch.device("mps")
        else: self.device = "cpu"

        self.demucs = models.Demucs(name="hdemucs_mmi", other_metadata={"segment":2, "split":True},
                        device=self.device, logger=None)
        

        self.count = 0 
        self.audio = []
        self.video_path = video_path
        
    
    def wav_audio_from_video(self):

        "Convert video to wav files"
        output_file = f"sample{self.count}.wav"
        display(Video(self.video_path,width=1024, height=576))
        self.audio.append(output_file)
        command = [
            "ffmpeg",
            "-i",self.video_path,
            "-q:a 0 -map a", #For Best quality
            output_file

        ]
        self.wav_file = output_file 
        subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=10**8)



    def classify_audio_make_sep_file(self):
        print("Generating Audio")

        res = self.demucs(self.wav_file) #Checked for audio_file
        for cls in res:
            print(f"Video File {self.video_path} Classified Class: {cls}")
            self.audio.append(f"{self.video_path}_V_{cls}.mp3")
            part = res[cls] 
            audiofile.write(f"cls_{cls}.mp3", part, self.demucs.sample_rate)
            display(Audio(f"cls_{cls}.mp3"))
        
    def display_audio(self):

        for i in self.audio:
            print(f"Audio {i}")
            Audio(i)



<h3>Files_listing Only MP4</h3>

In [187]:
def list_mp4_files_in_folder(folder_path):
    """
    Lists all .mp4 files in the specified folder.

    Args:
        folder_path (str): Path to the folder.

    Returns:
        list: A list containing the names of all .mp4 files in the folder.
    """
    try:
        # Get list of all files in the folder
        items = os.listdir(folder_path)
        # Filter for .mp4 files only
        mp4_files = [item for item in items if item.endswith('.mp4') and os.path.isfile(os.path.join(folder_path, item))]
        return mp4_files
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [189]:
files = list_mp4_files_in_folder('caught_behind')
files

['b_398_1_17_2_0_c1.mp4',
 'b_387_1_5_4_0_c1.mp4',
 'b_406_2_8_3_0_c1.mp4',
 'b_388_2_5_3_0_c1.mp4',
 'b_398_1_19_5_0_c1.mp4',
 'b_383_1_4_3_0_c1.mp4',
 'b_383_2_8_4_0_c1.mp4']

In [191]:
make = MakeAudio(f'caught_behind/{files[2]}')
make.wav_audio_from_video()
make.classify_audio_make_sep_file()

Generating Audio
Video File caught_behind/b_406_2_8_3_0_c1.mp4 Classified Class: drums


Video File caught_behind/b_406_2_8_3_0_c1.mp4 Classified Class: bass


Video File caught_behind/b_406_2_8_3_0_c1.mp4 Classified Class: other


Video File caught_behind/b_406_2_8_3_0_c1.mp4 Classified Class: vocals
